# WSI - laboratorium 6

Zadanie: Uczenie (się) ze wzmocnieniem

Autor: Jakub Mazurkiewicz (300226)

## Klasy pomocnicze

### Zestaw hiperparametrów

Poniższa klasa reprezentuje zestaw hiperparametrów:

In [2]:
from dataclasses import dataclass

@dataclass
class Hyperparameters:
    learning_rate: float = 0.1
    discount_factor: float = 0.6
    epsilon: float = 0.1
    episodes: int = 1000

    def __repr__(self):
        return '\n'.join([
            f'Learning rate: {self.learning_rate}',
            f'Discount factor: {self.discount_factor}',
            f'Epsilon: {self.epsilon}',
            f'Episodes: {self.episodes}'
        ])

### Wyniki uczenia

Poniższa klasa reprezentuje wyniki uczenia:

In [3]:
@dataclass
class LearningStats:
    penalty_count: int = 0
    total_reward: int = 0

### Wyniki ewaluacji